In [55]:
import pandas as pd
import sqlalchemy
from pathlib import Path
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [56]:
year = 2021

def file_reader_function(type, year):
    df = pd.read_csv(Path(f"../Resources/CRSS_data/{type}/{type}_{year}.csv"))
    return df

accident_df = file_reader_function("accident", year)
person_df = file_reader_function("person", year)
vehicle_df = file_reader_function("vehicle", year)

C:\Users\mirai\AppData\Local\Temp\ipykernel_3936\2647440939.py:4: DtypeWarning: Columns (55,57,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path(f"../Resources/CRSS_data/{type}/{type}_{year}.csv"))


In [57]:
accident_df = accident_df[["CASENUM", "MAN_COLL", "WEATHER", "MONTH", "HOUR", "DAY_WEEK", "ALCOHOL"]]
person_df = person_df[["CASENUM", "VEH_NO", "PER_NO", "AGE", "SEX", "DRUGS"]]
vehicle_df = vehicle_df[["CASENUM", "VEH_NO", "BODY_TYP", "SPEEDREL" ]]

# Join Vehicle and Person DataFrames to create "veh_and_per_df", left join to Vehicle DF. (Written by Ed)
veh_and_per_df = pd.merge(vehicle_df, person_df, on=['CASENUM','VEH_NO'], how='left')

# Join veh_and_per_df and accident df to create "allinfo_df", left join to veh_and_per_df. (Written by Ed)
data_df = pd.merge(veh_and_per_df,accident_df, on=['CASENUM'], how='left')

In [58]:
data_df = data_df[(data_df["PER_NO"] == 1)]
data_df = data_df.dropna(how="any")
data_df = data_df.drop(columns=["CASENUM", "VEH_NO", "PER_NO"])

In [59]:
data_df.head()

,BODY_TYP,SPEEDREL,AGE,SEX,DRUGS,MAN_COLL,WEATHER,MONTH,HOUR,DAY_WEEK,ALCOHOL
0,4,0,31.0,1.0,0.0,0,1,1,5,6,2
1,14,0,54.0,1.0,0.0,0,10,1,18,6,2
2,4,0,54.0,2.0,0.0,0,5,1,7,7,2
3,14,0,998.0,8.0,8.0,0,2,1,16,6,9
4,4,0,55.0,2.0,8.0,0,2,1,16,6,9


In [60]:
def drop_index_function(column_list, value_list):
    for columns in column_list:
        for values in value_list:
            drop_index = data_df[(data_df[columns] == values)].index
            data_df.drop(drop_index, inplace=True)

first_list = ['SEX', 'DAY_WEEK']
first_values = [8, 9]

drop_index_function(first_list, first_values)

second_list = ['MAN_COLL', 'WEATHER', 'HOUR']
second_values = [98, 99]

drop_index_function(second_list, second_values)

third_list = ['AGE']
third_values = [998, 999]

drop_index_function(third_list, third_values)

fourth_list = ['BODY_TYP']
fourth_values = [9, 19, 29, 39, 48, 49, 59, 71, 73, 89, 98, 99]

drop_index_function(fourth_list, fourth_values)

In [61]:
def replace_function(original, new):
    for row in original:
        data_df.loc[data_df["WEATHER"] == row] = new


weather_1 = [[3, 5, 6, 7], 8]
weather_2 = [[11], 4]
weather_3 = [[12], 2]

replace_function(weather_1[0], weather_1[1])
replace_function(weather_2[0], weather_2[1])
replace_function(weather_3[0], weather_3[1])

data_df["WEATHER"].unique()

array([ 1, 10,  8,  2,  4], dtype=int64)

In [62]:
data_df = data_df.reset_index(drop=True)
data_df["KEY"] = str(year) + data_df.index.astype(str)
column_reorder = data_df.columns.tolist()
column_reorder = column_reorder[-1:] + column_reorder[:-1]
data_df = data_df[column_reorder]

In [63]:
data_df["AGE"] = data_df["AGE"].astype(int)
data_df["SEX"] = data_df["SEX"].astype(int)
data_df["DRUGS"] = data_df["DRUGS"].astype(int)

In [64]:
data_df.head()

,KEY,BODY_TYP,SPEEDREL,AGE,SEX,DRUGS,MAN_COLL,WEATHER,MONTH,HOUR,DAY_WEEK,ALCOHOL
0,20210,4,0,31,1,0,0,1,1,5,6,2
1,20211,14,0,54,1,0,0,10,1,18,6,2
2,20212,8,8,8,8,8,8,8,8,8,8,8
3,20213,4,0,55,2,8,0,2,1,16,6,9
4,20214,14,0,59,2,8,0,2,1,16,6,9


In [65]:
data_df.columns

Index(['KEY', 'BODY_TYP', 'SPEEDREL', 'AGE', 'SEX', 'DRUGS', 'MAN_COLL',
       'WEATHER', 'MONTH', 'HOUR', 'DAY_WEEK', 'ALCOHOL'],
      dtype='object')

In [66]:
# Creating SQL Alchemy Engine and connection to engine
database_path = ("FlaskApp/Accidents_Database.db")
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [423]:
data_df.to_sql("accidents_data", conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

80687